# Configuration

For stockstats lib, see this github repo https://github.com/jealous/stockstats/blob/master/stockstats.py

In [ ]:
%pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 KB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 KB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.2/111.2 KB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 KB 4.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.25.1
    Uninstalling requests-2.25.1:
      Successfully uninstalled requests-2.25.1
  Attempting uninstall: html5lib
    Found existing installation: html5lib 1.0.1
    Uninstalling html5lib-1.0.1:
      Successfully uninstalled html5lib-1.0.1
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
   

In [ ]:
%pip install stockstats

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


If the below command doesn't work by not having the otpion --use-deprecated=backtrack-on-build-failures, try to downgrade pip to 22.0.4 version

In [ ]:
%pip install npzigzag --use-deprecated=backtrack-on-build-failures

DEPRECATION: Backtracking on build failures can mask issues related to how a package generates metadata or builds a wheel. This flag will be removed in pip 22.2. A possible replacement is avoiding known-bad versions by explicitly telling pip to ignore them (either directly as requirements, or via a constraints file). Discussion can be found at https://github.com/pypa/pip/issues/10655
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 26.1 MB/s eta 0:00:00
  Created wheel for npzigzag: filename

In [ ]:
!pip install matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
import yfinance as yf
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import numpy as np
from stockstats import wrap
from stockstats import StockDataFrame
from npzigzag import core as zz
import plotly.graph_objects as go

c:\Users\005\anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Business Logic

## Plot

In [2]:
def price_plot(price):
  assert type(price) == pd.core.frame.DataFrame, "The input price dataframe should be a pandas dataframe"
  assert "Close" in price.columns, "The price dataframe should have a Close pricing column"
  assert "Dividends" in price.columns, "The price dataframe should have a Dividends column"

  price["Closing Div Adjusted"] = price["Close"] + price["Dividends"]
  fig = px.line(x=price.index, y=price["Closing Div Adjusted"], title="Price Plot")
  fig.show()

In [3]:
def macd_plot(price, attr={}, show_plot=True):
    # Pandas variable denote whether input is pandas dataframe or not
    # macd = pd.DataFrame(wrap(price)["macd"])
    # fig = px.line(x=macd.index, y=macd["macd"])
    # fig.show()
    # return macd
    stockstats_price = wrap(price.copy())
    for k, v in attr.items():
        setattr(stockstats_price, k, v)
    stockstats_price.init_all()

    macd = pd.DataFrame(stockstats_price["macd"])
    fig = px.line(x=macd.index, y=macd["macd"])
    if show_plot:
        fig.show()
    return macd


In [4]:
def macd_details_plot(price, attr={}):
  #Ray has provided another logic which is a bit different from the one in stockstats
  #Ray's DIFF is stockstats's macd, DEA is macds, MACD is 2*macdh
  
  price_in_stockstats = wrap(price.copy())
  for k, v in attr.items():
    setattr(price_in_stockstats, k, v)
  price_in_stockstats.init_all()

  price_in_stockstats["2xmacdh"] = 2*price_in_stockstats["macdh"]
  fig = px.line(price_in_stockstats, x=price_in_stockstats.index, y=["macd", "macds", "2xmacdh"], title="MACD Plot")
  newnames = {'macd':'DIFF', 'macds': 'DEA', "2xmacdh":"macd"}
  fig.for_each_trace(lambda t: t.update(name = newnames[t.name],
                                        legendgroup = newnames[t.name],
                                        hovertemplate = t.hovertemplate.replace(t.name, newnames[t.name])))

  fig.show()

In [5]:
def zigzag_plot(stock_price, show_plot=True):
  zz_pivots = zz.zigzag(stock_price['Close'], 0.009, True)
  zig_zag = np.interp(x=stock_price.index, xp=zz_pivots, fp=stock_price['Close'][zz_pivots])

  fig = px.line(x=stock_price.index, y=[stock_price['Close'], zig_zag], title="Price Plot with ZigZag") 
  label_name = {'wide_variable_0': 'Price', 'wide_variable_1': 'Zig Zag'}
  fig.for_each_trace(lambda t: t.update(name = label_name[t.name],
                                          legendgroup = label_name[t.name],
                                          hovertemplate = t.hovertemplate.replace(t.name, label_name[t.name])))
  if (show_plot):
    fig.show()
  return zz_pivots

In [6]:
def macd_details_date_stripped_plot(price, attr={}):
  #Ray has provided another logic which is a bit different from the one in stockstats
  #Ray's DIFF is stockstats's macd, DEA is macds, MACD is 2*macdh

  #Strip away the datetime property by converting the datetime index into string
  #Useful for hourly data where there is a large and disproprotional gap between two trading days
  #It has hardcoded for hourly by using 7 as the step. And if a trading day has only half day and not 7 hourly data, this could break
  #Should only be used for plotting and not for getting macd data
  
  price_in_stockstats = wrap(price.copy())
  for k, v in attr.items():
    setattr(price_in_stockstats, k, v)
  price_in_stockstats.init_all()

  datetime_string_series = price_in_stockstats.index.strftime("%Y-%m-%d %H:%M NYC").to_series()
  price_in_stockstats["2xmacdh"] = 2*price_in_stockstats["macdh"]
  fig = px.line(price_in_stockstats, x=datetime_string_series, y=["macd", "macds", "2xmacdh"], title="MACD Plot")
  newnames = {'macd':'DIFF', 'macds': 'DEA', "2xmacdh":"macd"}
  fig.for_each_trace(lambda t: t.update(name = newnames[t.name],
                                        legendgroup = newnames[t.name],
                                        hovertemplate = t.hovertemplate.replace(t.name, newnames[t.name])))
  

  tickvals_list = []
  ticktext_list = []

  for i in range(0, len(price_in_stockstats.index), 7):
    tickvals_list.append(datetime_string_series[i])
    ticktext_list.append(price_in_stockstats.index[i].strftime("%Y-%m-%d %H:%M"))

  fig.update_xaxes(tickvals = tickvals_list, ticktext = ticktext_list)

  fig.show()

In [7]:
def zigzag_date_stripped_plot(stock_price):
  datetime_string_index = stock_price.index.strftime("%Y-%m-%d %H:%M NYC").to_series()
  stock_price = stock_price.set_index(pd.Series(range(len(stock_price.index))))
  zz_pivots = zz.zigzag(stock_price['Close'], 0.009, True)
  zig_zag = np.interp(x=stock_price.index, xp=zz_pivots, fp=stock_price['Close'][zz_pivots])
  stock_price = stock_price.set_index(datetime_string_index)

  fig = px.line(x=stock_price.index, y=[stock_price['Close'], zig_zag], title="Price Plot with ZigZag") 
  label_name = {'wide_variable_0': 'Price', 'wide_variable_1': 'Zig Zag'}
  fig.for_each_trace(lambda t: t.update(name = label_name[t.name],
                                          legendgroup = label_name[t.name],
                                          hovertemplate = t.hovertemplate.replace(t.name, label_name[t.name])))
  fig.show()

In [8]:
def zigzag_plot_with_slider(stock_price):
  # Create figure
  fig = go.Figure()

  # Add traces, one for each slider step
  for step in np.arange(0, 0.05, 0.001):
      zz_pivots = zz.zigzag(stock_price['Close'], step, True)
      zig_zag = np.interp(x=stock_price.index, xp=zz_pivots, fp=stock_price['Close'][zz_pivots])
      fig.add_trace(
          go.Line(
              visible=False,
              name='Price',
              x=stock_price.index,
              y=stock_price['Close']))
      fig.add_trace(
          go.Line(
              visible=False,
              name='Zig Zag',
              x=stock_price.index,
              y=zig_zag))    

  # Make 10th trace visible
  # fig.data[-1].visible = True

  # Create and add slider
  steps = []
  for i in range(len(fig.data) - 1):
      if i % 2 == 0:
          step = dict(
              method="update",
              args=[{"visible": [False] * len(fig.data)},
                  {"title": "Slider switched to step: " + str(i)}],  # layout attribute
          )
          step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
          step["args"][0]["visible"][i + 1] = True
          steps.append(step)

  sliders = [dict(
      active=0,
      currentvalue={"prefix": "Frequency: "},
      pad={"t": 50},
      steps=steps
  )]

  fig.update_layout(
      sliders=sliders
  )


  fig.show()

# Run

In [9]:
stock = yf.Ticker("BABA")
stock_price = stock.history(period="7mo", interval='1d')

## Day

In [10]:
price_plot(stock_price)

In [11]:
macd_details_plot(stock_price)

Drag the slider to choose different fitness of the zigzag

In [12]:
zigzag_plot_with_slider(stock_price)

c:\Users\005\anaconda3\lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




## Hourly

In [13]:
stock_price_hr = stock.history(period="151d", interval='1h') # last interval of a day only 30 minutes

In [14]:
MACD_EMA_SHORT = 12
MACD_EMA_LONG = 26
MACD_EMA_SIGNAL = 9

macd_details_date_stripped_plot(stock_price_hr, attr={'MACD_EMA_SHORT': MACD_EMA_SHORT, 'MACD_EMA_LONG': MACD_EMA_LONG, 'MACD_EMA_SIGNAL': MACD_EMA_SIGNAL})

In [15]:
zigzag_date_stripped_plot(stock_price_hr);

# Core Logic Dev

## Core Logic Methods

In [16]:
def find_points_diff(price, show_error=False):
    """
    args: price---- Raw price data
    """
    diff_line = macd_plot(price.copy(), show_plot=False)['macd']
    diff_line_deriv = np.diff(diff_line.to_numpy())
    diff_line_deriv = np.append(np.sign(diff_line_deriv[:1]), diff_line_deriv)
    diff_line_sgn_change = np.append(((np.diff(np.sign(diff_line_deriv)) > 0)*1), np.zeros(1)).astype(bool)

    diff_line_low = diff_line[diff_line_sgn_change]
    if not np.all(diff_line_low):
      diff_line_low = diff_line[np.logical_and(diff_line_sgn_change, (diff_line == 0))]
      if show_error:
        print()
        print("We got a 0 division here!!!!!!")
        print(diff_line)
        print()
        print("Above is diff line (original macd); below is diff low")
        print()
        print(diff_line_low)
        print("We will remove such macd with 0 so that no zero division. The possible reason is when there are two adjacent time having the same price in the beginning of the df, then macd will have two consecutive 0s. This likely happens in 15m or even finer-time-granular data")
        print()
    diff_line_divided = np.append(np.zeros(1).astype(bool), np.divide(np.diff(diff_line_low), np.abs(diff_line_low[:-1])) >= 0.2)
    diff_line_signal = diff_line_low[diff_line_divided]

    close = price['Close']
    price_at_diff_low = close[diff_line_sgn_change]
    price_lower_indicator = np.diff(price_at_diff_low, prepend=price_at_diff_low[0]) < 0
    signal = price_at_diff_low[np.logical_and(diff_line_divided, price_lower_indicator)]

    return signal


In [17]:
def validate_macd_signal(price, signal, show_print=False):
  if signal.empty:
    if show_print:
      print("signal is empty")
    return False

  macd_signal_date = signal.index[-1]
  stock_price_at_and_after_signal = price.loc[price.index >= macd_signal_date]
  stock_price_after_signal = stock_price_at_and_after_signal[1:]
  # We left a 0.5% fuzzy buffer for the low since it's very possible that a price low point happens before the macd point candidate, 
  # and the macd candidate's corresponding price is actually not low. Ray said it makes more sense to look at the low price near and before
  # the macd candidate. As an expedient, we use 0.5% so that we don't overkill a good point in the validation. 
  # Moreover, we can also look at the hourly close low of the day of macd candidate
  low_after_signal = np.min(stock_price_after_signal["Close"]) * 1.005
  new_low_after_signal = low_after_signal < stock_price_at_and_after_signal.iloc[0]["Close"]
  if new_low_after_signal:
    if show_print:
      print("new price low " + str(np.min(stock_price_after_signal["Close"])) + 
          " occurs after macd signal; macd signal is invalid if a data point after it has lower price than the price at macd signal date")
      print("macd candidate close price is " + str(stock_price_at_and_after_signal.iloc[0]["Close"]))
    
    hourly_low_at_macd_signal_day = np.min(stock_price_hr[stock_price_hr.index.strftime("%Y-%m-%d") == signal.index[-1].strftime("%Y-%m-%d")]["Close"])
    
    if show_print:
        print("The minimum of hourly low at macd_candidate is " + str(hourly_low_at_macd_signal_day))
    if low_after_signal < hourly_low_at_macd_signal_day:
      return False

  return True

In [18]:
def find_high_and_low_pivots(price):
  pivots_indices = zigzag_plot(price.copy(), False)
  
  if (len(pivots_indices) < 4):
    raise Exception("why pivots length is less than 3? Input stock price data too small")

  odd_indices = range(1, len(pivots_indices), 2)
  even_indices = range(2, len(pivots_indices), 2)

  price_pivots_df = price.loc[pivots_indices]

  pivots = {}
  
  if price_pivots_df["Close"][1] < price_pivots_df["Close"][2]:
    pivots["low"] = price_pivots_df.iloc[odd_indices]
    pivots["high"] = price_pivots_df.iloc[even_indices]
  else:
    pivots["high"] = price_pivots_df.iloc[odd_indices]
    pivots["low"] = price_pivots_df.iloc[even_indices]
  return pivots

In [19]:
def find_buy_points(price, signal):
  pivots = find_high_and_low_pivots(price)
  high_pivots = pivots["high"]
  low_pivots = pivots["low"]
  marked_high = high_pivots.loc[high_pivots.index >= signal.index[-1]]
  if marked_high.empty:
    raise Exception("No high point after signal, continue")
  marked_high = marked_high.iloc[0]
  # print("marked high point after macd signal:")
  # print(marked_high)

  low_pivots_after_signal = low_pivots.loc[low_pivots.index >= signal.index[-1]]
  #回踩不入
  fall_but_not_below = low_pivots_after_signal.loc[low_pivots_after_signal["Close"] > marked_high["Close"]]
  if fall_but_not_below.empty:
    raise Exception("No buy points")
  return fall_but_not_below.iloc[0]

In [20]:
def macd_zigzag_plot(price):
  macd_details_date_stripped_plot(price)
  zigzag_date_stripped_plot(price)

In [21]:
class Stock_Point:
  stop_loss_coeff_default = 0.94
  def __init__(self, time_index, close_price):
    self.time_index = time_index
    self.original_close_price = close_price
    self.next_stop_loss_rise_price_threshold = close_price
    self.stop_loss = close_price * Stock_Point.stop_loss_coeff_default
    self.stop_loss_coeff = Stock_Point.stop_loss_coeff_default
    self.stop_loss_coeff_adjust_price_threshold = close_price * 1.1
    self.previous_partial_sell_return = None
    self.previous_partial_sell_percentage = None

  def get_return(self, new_price):
    current_return = (new_price - self.original_close_price) / self.original_close_price
    if (self.previous_partial_sell_return == None):
      return current_return
    else:
      return (1 - self.previous_partial_sell_percentage) * current_return + self.previous_partial_sell_percentage * self.previous_partial_sell_return
    

  def update_and_should_sell(self, new_price, new_price_date, stop_loss_schema="vanilla"):
    if stop_loss_schema == "vanilla":
      return self.update_and_should_sell_vanilla(new_price)

    if stop_loss_schema == "half_sell_half_move":
      return self.update_and_should_sell_hal_sell_half_move(new_price, new_price_date)

    raise Exception("The input stop loss schema is not found")

  def update_and_should_sell_hal_sell_half_move(self, new_price, new_price_date, half_sell_threshold=1.05):
    if ((self.previous_partial_sell_return == None) and (new_price > self.original_close_price * half_sell_threshold)):
      self.previous_partial_sell_return = (new_price - self.original_close_price) / self.original_close_price
      self.previous_partial_sell_percentage = 0.5
      print("We sell half of the trade we bought at " + str(self.time_index) + " under price " + str(self.original_close_price) + " at the date " + str(new_price_date) + " under price " + str(new_price))

    if (new_price < self.stop_loss):
      return True
    else:
      if (self.next_stop_loss_rise_price_threshold < new_price):
        if (self.stop_loss_coeff_adjust_price_threshold < new_price):
          self.stop_loss_coeff_adjust_price_threshold = self.stop_loss_coeff_adjust_price_threshold * 1.1
          self.stop_loss_coeff = max(0.9, self.stop_loss_coeff - 0.01)

        self.next_stop_loss_rise_price_threshold = new_price
        self.stop_loss = new_price * self.stop_loss_coeff
      return False

  def update_and_should_sell_vanilla(self, new_price, new_price_date):
    if (new_price < self.stop_loss):
      return True
    else:
      if (self.next_stop_loss_rise_price_threshold < new_price):
        self.next_stop_loss_rise_price_threshold = new_price
        self.stop_loss = new_price * self.stop_loss_coeff
      return False


## Single Set

In [ ]:
stock = yf.Ticker("QQQ")
stock_price_hr = stock.history(start="2022-12-15", end="2023-01-30", interval='1h') # last interval of a day only 30 minutes
macd_signal = find_points_diff(stock_price_hr)
macd_signal

Series([], Name: Close, dtype: float64)

In [ ]:
macd_details_date_stripped_plot(stock_price_hr)
zigzag_date_stripped_plot(stock_price_hr)

In [ ]:
validate_macd_signal(stock_price_hr, macd_signal)

True

In [ ]:
find_buy_points(stock_price_hr, macd_signal)

Exception: ignored

## Multi Set with a Window Size

In [ ]:
stock = yf.Ticker("TSLA")
stock_price_hr = stock.history(period="60d", interval='15m') # last interval of a day only 30 minutes
#stock_price_hr = stock_price_hr[np.logical_and(stock_price_hr.index >= "2022-06-01", stock_price_hr.index <= "2023-12-15")]

In [ ]:
# sliding window
sets_checked = 0
window_size = 21*7
real_trade_point_and_buy_point_max_time_difference = 3*7
last_buy_point_dataset = None
buy_points = {}

for i in range(len(stock_price_hr)-window_size):
    segment = stock_price_hr.iloc[i:i+window_size, :].copy()

    signal = find_points_diff(segment)

    if validate_macd_signal(segment, signal, False):
      try:
        buy_point = find_buy_points(segment, signal)

        today_df_loc = segment.index.get_loc(segment.index[-1])
        buy_point_loc = segment.index.get_loc(buy_point.name)

        if (today_df_loc - buy_point_loc) < real_trade_point_and_buy_point_max_time_difference:

          print("We start at time " + str(segment.index[0]))
          print("We are now at time " + str(segment.index[-1]))
          print(signal)
          print("Below is buy point")
          print(buy_point)
          print()

          if buy_point.name not in buy_points.keys():
            buy_points[buy_point.name] = [segment.index[-1]]
          else:
            buy_points[buy_point.name].append(segment.index[-1])

          last_buy_point_dataset = segment
      except:
        pass
    
    sets_checked += 1
    if(sets_checked % 210 == 0):
      print("We have checked " + str(sets_checked) + " sets")
      print()

We have checked 210 sets

We start at time 2022-12-20 14:45:00-05:00
We are now at time 2022-12-29 12:15:00-05:00
Datetime
2022-12-23 10:00:00-05:00    121.769997
2022-12-27 15:45:00-05:00    109.099998
Name: Close, dtype: float64
Below is buy point
Open            1.220018e+02
High            1.220600e+02
Low             1.201600e+02
Close           1.201621e+02
Volume          6.696225e+06
Dividends       0.000000e+00
Stock Splits    0.000000e+00
Name: 2022-12-29 12:00:00-05:00, dtype: float64

We start at time 2022-12-20 15:00:00-05:00
We are now at time 2022-12-29 12:30:00-05:00
Datetime
2022-12-23 10:00:00-05:00    121.769997
2022-12-27 15:45:00-05:00    109.099998
Name: Close, dtype: float64
Below is buy point
Open            1.220018e+02
High            1.220600e+02
Low             1.201600e+02
Close           1.201621e+02
Volume          6.696225e+06
Dividends       0.000000e+00
Stock Splits    0.000000e+00
Name: 2022-12-29 12:00:00-05:00, dtype: float64

We start at time 2022-

KeyboardInterrupt: ignored

In [ ]:
macd_zigzag_plot(last_buy_point_dataset)

### Backtest

In [ ]:
real_trade_buy_points = []
first_real_trade_buy_points_time_index = None
trade_returns = []
avg_return = None

for key in buy_points.keys():
  buy_point_found_dates = buy_points[key]
  if len(buy_point_found_dates) > 1:
    real_trade_buy_point_time_index = buy_point_found_dates[1]
    real_trade_buy_points.append(Stock_Point(real_trade_buy_point_time_index, stock_price_hr.loc[real_trade_buy_point_time_index]["Close"]))

if len(real_trade_buy_points) > 0:
  first_real_trade_buy_points_time_index = real_trade_buy_points[0].time_index
  stock_price_after_trade = stock_price_hr[stock_price_hr.index > first_real_trade_buy_points_time_index]

  for i in range(len(stock_price_hr)):
    current_day_price = stock_price_hr.iloc[i, :]
    real_trade_buy_points_copy = real_trade_buy_points[:]
    for trade_point in real_trade_buy_points:
      if (current_day_price.name > trade_point.time_index):
        should_sell = trade_point.update_and_should_sell(current_day_price["Close"])
        if should_sell or (i == (len(stock_price_hr) - 1)):
          trade_returns.append((current_day_price["Close"] - trade_point.original_close_price) / trade_point.original_close_price)
          print("We sell the trade we bought at " + str(trade_point.time_index) + " under price " + str(trade_point.original_close_price) + " at the date " + str(current_day_price.name) + " under price " + str(current_day_price["Close"]))
          real_trade_buy_points_copy.remove(trade_point)
    real_trade_buy_points = real_trade_buy_points_copy

if len(trade_returns) > 0:
  avg_return = sum(trade_returns)/len(trade_returns)

avg_return

We sell the trade we bought at 2021-10-25 22:00:00+00:00 under price 63041.73828125 at the date 2021-10-27 09:00:00+00:00 under price 58936.265625
We sell the trade we bought at 2021-10-26 16:00:00+00:00 under price 62029.09765625 at the date 2021-10-27 11:00:00+00:00 under price 58603.2890625
We sell the trade we bought at 2021-11-23 17:00:00+00:00 under price 57314.05078125 at the date 2021-11-26 08:00:00+00:00 under price 54999.0
We sell the trade we bought at 2021-12-16 00:00:00+00:00 under price 49108.94921875 at the date 2021-12-17 14:00:00+00:00 under price 45952.53125
We sell the trade we bought at 2022-01-29 18:00:00+00:00 under price 37854.96484375 at the date 2022-02-03 10:00:00+00:00 under price 36449.54296875
We sell the trade we bought at 2022-02-05 18:00:00+00:00 under price 41638.7421875 at the date 2022-02-11 20:00:00+00:00 under price 42484.98046875
We sell the trade we bought at 2022-02-06 16:00:00+00:00 under price 41643.9453125 at the date 2022-02-11 20:00:00+00:00

0.008243810401929636

## Bitcoin with Binance for 15m and more data (Deprecated)

In [ ]:
import requests
import json
import datetime as dt

In [ ]:
url = 'https://api.binance.us/api/v3/klines'
symbol = 'BTCUSD'
interval = '15m'
start = str(int(dt.datetime(2018,1,1).timestamp()*1000))
end = str(int(dt.datetime(2019,12,30).timestamp()*1000))
par = {'symbol': symbol, 'interval': interval, 'startTime': start, 'endTime': end}
data = pd.DataFrame(json.loads(requests.get(url, params= par).text))
#format columns name
data.columns = ['datetime', 'open', 'high', 'low', 'Close', 'volume','close_time', 'qav', 'num_trades','taker_base_vol', 'taker_quote_vol', 'ignore']
data.index = [dt.datetime.fromtimestamp(x/1000.0) for x in data.datetime]
data=data.astype(float)

In [ ]:
data

,datetime,open,high,low,Close,volume,close_time,qav,num_trades,taker_base_vol,taker_quote_vol,ignore
2019-09-17 10:15:00,1.568715e+12,10170.0,10170.0,10170.0,10170.0,0.002,1.568716e+12,20.34,1.0,0.0,0.0,0.0
2019-09-17 10:30:00,1.568716e+12,10170.0,10170.0,10170.0,10170.0,0.000,1.568717e+12,0.00,0.0,0.0,0.0,0.0
2019-09-17 10:45:00,1.568717e+12,10170.0,10170.0,10170.0,10170.0,0.000,1.568718e+12,0.00,0.0,0.0,0.0,0.0
2019-09-17 11:00:00,1.568718e+12,10170.0,10170.0,10170.0,10170.0,0.000,1.568719e+12,0.00,0.0,0.0,0.0,0.0
2019-09-17 11:15:00,1.568719e+12,10170.0,10170.0,10170.0,10170.0,0.000,1.568720e+12,0.00,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2019-09-22 14:00:00,1.569161e+12,10170.0,10170.0,10170.0,10170.0,0.000,1.569162e+12,0.00,0.0,0.0,0.0,0.0
2019-09-22 14:15:00,1.569162e+12,10170.0,10170.0,10170.0,10170.0,0.000,1.569163e+12,0.00,0.0,0.0,0.0,0.0
2019-09-22 14:30:00,1.569163e+12,10170.0,10170.0,10170.0,10170.0,0.000,1.569163e+12,0.00,0.0,0.0,0.0,0.0
2019-09-22 14:45:00,1.569164e+12,10170.0,10170.0,10170.0,10170.0,0.000,1.569164e+12,0.00,0.0,0.0,0.0,0.0


In [ ]:
len(json.loads(requests.get(url, params= par).text))

500

In [ ]:
stock = yf.Ticker("BTC-USD")
stock_price_hr = stock.history(start="2023-01-01", end="2023-01-07", interval='15m') # last interval of a day only 30 minutes
#macd_zigzag_plot(stock_price_hr)
stock_price_hr

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Datetime,,,,,,,
2023-01-01 00:00:00+00:00,16547.914062,16553.699219,16543.115234,16543.115234,10700800,0.0,0.0
2023-01-01 00:15:00+00:00,16543.712891,16551.250000,16535.767578,16551.250000,3176448,0.0,0.0
2023-01-01 00:30:00+00:00,16550.699219,16551.419922,16530.056641,16530.056641,23931904,0.0,0.0
2023-01-01 00:45:00+00:00,16530.537109,16539.396484,16529.074219,16539.396484,8493056,0.0,0.0
2023-01-01 01:00:00+00:00,16538.013672,16546.175781,16536.416016,16545.587891,0,0.0,0.0
...,...,...,...,...,...,...,...
2023-01-06 22:45:00+00:00,16946.367188,16958.109375,16946.367188,16958.042969,41656320,0.0,0.0
2023-01-06 23:00:00+00:00,16957.191406,16963.695312,16951.033203,16963.009766,10290176,0.0,0.0
2023-01-06 23:15:00+00:00,16964.605469,16971.310547,16964.605469,16967.076172,25705472,0.0,0.0


In [ ]:
data

,datetime,open,high,low,Close,volume,close_time,qav,num_trades,taker_base_vol,taker_quote_vol,ignore
2023-01-01 00:00:00,1.672531e+12,16526.26,16527.58,16500.87,16503.29,34.237552,1.672532e+12,5.652234e+05,625.0,14.812094,244539.3756,0.0
2023-01-01 00:15:00,1.672532e+12,16502.97,16523.90,16500.00,16521.44,38.618154,1.672533e+12,6.378280e+05,641.0,16.791596,277320.7290,0.0
2023-01-01 00:30:00,1.672533e+12,16521.59,16523.84,16498.49,16507.19,54.564954,1.672534e+12,9.008587e+05,1186.0,16.893331,278912.8677,0.0
2023-01-01 00:45:00,1.672534e+12,16503.65,16522.92,16503.65,16516.48,16.280429,1.672535e+12,2.689169e+05,398.0,9.694607,160139.9474,0.0
2023-01-01 01:00:00,1.672535e+12,16516.08,16526.50,16513.23,16523.84,23.010349,1.672536e+12,3.801682e+05,550.0,12.757644,210783.4254,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-06 03:45:00,1.672977e+12,16832.95,16835.53,16825.82,16827.00,15.667790,1.672978e+12,2.637012e+05,334.0,5.736552,96554.6409,0.0
2023-01-06 04:00:00,1.672978e+12,16827.64,16831.53,16824.36,16827.31,10.418415,1.672978e+12,1.753178e+05,475.0,4.841933,81481.4477,0.0
2023-01-06 04:15:00,1.672978e+12,16827.20,16827.55,16802.84,16805.00,66.071996,1.672979e+12,1.110948e+06,1085.0,26.643528,447953.8821,0.0
2023-01-06 04:30:00,1.672979e+12,16806.34,16816.08,16800.23,16810.84,20.660770,1.672980e+12,3.473077e+05,1167.0,9.409792,158186.3900,0.0


In [ ]:
zigzag_plot(stock_price_hr)

DatetimeIndex(['2023-01-01 00:00:00+00:00', '2023-01-04 04:30:00+00:00'], dtype='datetime64[ns, UTC]', name='Datetime', freq=None)

In [ ]:
zigzag_plot(data)

DatetimeIndex(['2023-01-01'], dtype='datetime64[ns]', freq=None)

## Adapted FastQuant Bitcoin data

In [ ]:
!pip3 install ccxt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 50.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.0/289.0 KB 28.9 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.9.0 requires jedi>=0.10, which is not installed.
tensorflow 2.11.0 requires numpy>=1.20, but you have numpy 1.19.1 which is incompatible.
cvxpy 1.2.3 requires setuptools<=64.0.2, but you have setuptools 67.5.1 which is incompatible.


In [22]:
#adapted from fastquant get_crypto_data

from datetime import datetime, timedelta
import pandas as pd
import ccxt

# Only support top 6 listed on https://www.coingecko.com/en/exchanges for now
CRYPTO_EXCHANGES = [
    "binanceus",
    "coinbasepro",
    "bithumb",
    "kraken",
    "kucoin",
    "bitstamp",
]
# to add more just add more method names to the above
# list of supported exchanges according to the classes mentioned here: https://github.com/ccxt/ccxt/tree/master/python/ccxt

DATETIME_FORMAT = {"daily": "%Y-%m-%d", "intraday": "%Y-%m-%d %H:%M:%S"}


def unix_time_millis(date):
    # epoch = datetime.utcfromtimestamp(0)

    # value will only have : if the date passed is intraday
    dt_format = DATETIME_FORMAT["intraday"] if ":" in date else DATETIME_FORMAT["daily"]
    dt = datetime.strptime(date, dt_format)
    # return int((dt - epoch).total_seconds() * 1000)
    return int(dt.timestamp() * 1000)


def get_crypto_data(
    ticker, start_date, end_date, time_resolution="1d", exchange="binanceus"
):
    """
    Get crypto data in OHLCV format

    Parameters
    ----------
    ticker : str
        List of ticker symbols here: https://coinmarketcap.com/exchanges/binance/
    start_date, end_date : str
        date in YYYY-MM-DD format
    time_resolution : str
       resolutions: '1w', '1d' (default), '1h', '1m'
    exchange : str
       market exchanges: 'binance' (default), 'coinbasepro', 'bithumb', 'kraken', 'kucoin', 'bitstamp'
    """
    dt_format = (
        DATETIME_FORMAT["intraday"]
        if "m" in time_resolution or "h" in time_resolution
        else DATETIME_FORMAT["daily"]
    )
    start_date_epoch = unix_time_millis(start_date)
    end_date_epoch = unix_time_millis(end_date)

    if exchange in CRYPTO_EXCHANGES:
        # Get the exchange we want to use from ccxt's exchange attributes
        ex = getattr(ccxt, exchange)({"verbose": False})

        # We're going to get data in batches, so want to know what the last record in the previous batch was
        previous_request_end_date_epoch = start_date_epoch

        # The time we want data from will shift each batch as well
        request_start_date_epoch = start_date_epoch

        # Variable to store our dataframe as we fill it out
        ohlcv_df = None

        while previous_request_end_date_epoch < end_date_epoch:
            # Pull the data from the exchange
            ohlcv_lol = ex.fetch_ohlcv(
                ticker, time_resolution, since=request_start_date_epoch
            )
            # Convert it to a dataframe
            current_request_df = pd.DataFrame(
                ohlcv_lol,
                columns=["dt", "open", "high", "low", "close", "volume"],
            )

            if current_request_df.size == 0:
                # If we got no results (which happens sometimes, like on binance for ETH/BTC when requesting 2018-02-08)
                # then step forward to the next day
                request_start_date_epoch += (
                    int(timedelta(days=1).total_seconds()) * 1000
                )
                # Make sure we're at the start of that day
                request_start_date_epoch = unix_time_millis(
                    pd.to_datetime(request_start_date_epoch, unit="ms").strftime(
                        dt_format
                    )
                )
                previous_request_end_date_epoch = request_start_date_epoch - 1
                continue

            if ohlcv_df is None:
                # We don't have a dataframe yet, so start with this
                ohlcv_df = current_request_df
            else:
                # Trim any overlap with the new results
                ohlcv_df = ohlcv_df[ohlcv_df.dt < current_request_df.dt.min()]
                # Append the results to what we have so far
                ohlcv_df = ohlcv_df.append(current_request_df)

            # Get the last entry timestamp after we've retrieved (or attempted to) additional records
            current_request_end_date_epoch = int(ohlcv_df.dt.max())

            if current_request_end_date_epoch <= previous_request_end_date_epoch:
                # We haven't gained any additional records, so there's no point in further requests
                # Let's mark this for the data end date, mostly so both end_date and end_date_epoch will be
                # in sync in case someone in future uses them in code futher down and to ensure the loop bails
                end_date_epoch = current_request_end_date_epoch
                # Update the actual end date so that the stored value will reflect the actual end
                end_date = pd.to_datetime(end_date_epoch, unit="ms")
                # The loop would exit based on the end_date_epoch value, but we'll save that check occuring
                break
            else:
                # We've gained some more records, so let's place another request (unless we're past the end, but our loop will catch that without checking here)
                # The next request should start a millisecond after this one ended
                request_start_date_epoch = current_request_end_date_epoch + 1
                # This request's end date should now be set as current for the next loop
                previous_request_end_date_epoch = current_request_end_date_epoch

        if ohlcv_df is not None:
            # Convert the unix timestampe to datetime
            ohlcv_df["dt"] = pd.to_datetime(ohlcv_df["dt"], unit="ms")
            # Trim off any records which were returned beyond the end
            ohlcv_df = ohlcv_df[ohlcv_df.dt <= end_date]
            # Save input parameters into dataframe
            ohlcv_df.start_date = start_date
            ohlcv_df.end_date = end_date
            ohlcv_df.symbol = ticker
            ohlcv_df = ohlcv_df.set_index("dt")

        return ohlcv_df
    else:
        raise NotImplementedError(
            "The exchange " + exchange + " is not yet supported. Available exchanges: "
            ", ".join(CRYPTO_EXCHANGES)
        )


In [23]:
crypto = get_crypto_data("BTC/USDT", "2019-09-25", "2023-02-28", time_resolution="1h")
crypto = crypto.rename(columns={"close":"Close"})
crypto

,open,high,low,Close,volume
dt,,,,,
2019-09-25 07:00:00,8477.23,8478.96,8331.71,8344.39,7.088447
2019-09-25 08:00:00,8329.58,8385.97,8257.35,8266.16,3.382833
2019-09-25 09:00:00,8270.50,8270.50,8270.50,8270.50,0.056111
2019-09-25 10:00:00,8403.67,8451.57,8391.96,8405.43,1.038796
2019-09-25 11:00:00,8432.68,8432.68,8314.48,8329.21,0.560000
...,...,...,...,...,...
2023-02-27 20:00:00,23270.59,23329.00,23117.11,23321.49,101.130309
2023-02-27 21:00:00,23318.94,23393.06,23299.86,23379.92,26.192011
2023-02-27 22:00:00,23382.44,23559.53,23362.32,23477.97,61.863232


In [26]:
stock_price_hr = crypto
stock_price_hr

,open,high,low,Close,volume
dt,,,,,
2019-09-25 07:00:00,8477.23,8478.96,8331.71,8344.39,7.088447
2019-09-25 08:00:00,8329.58,8385.97,8257.35,8266.16,3.382833
2019-09-25 09:00:00,8270.50,8270.50,8270.50,8270.50,0.056111
2019-09-25 10:00:00,8403.67,8451.57,8391.96,8405.43,1.038796
2019-09-25 11:00:00,8432.68,8432.68,8314.48,8329.21,0.560000
...,...,...,...,...,...
2023-02-27 20:00:00,23270.59,23329.00,23117.11,23321.49,101.130309
2023-02-27 21:00:00,23318.94,23393.06,23299.86,23379.92,26.192011
2023-02-27 22:00:00,23382.44,23559.53,23362.32,23477.97,61.863232


In [25]:
# sliding window
sets_checked = 0
window_size = 21*24
real_trade_point_and_buy_point_max_time_difference = 3*24
last_buy_point_dataset = None
buy_points = {}

for i in range(len(stock_price_hr)-window_size):
    segment = stock_price_hr.iloc[i:i+window_size, :].copy()

    signal = find_points_diff(segment, True)

    if validate_macd_signal(segment, signal, False):
      try:
        buy_point = find_buy_points(segment, signal)

        today_df_loc = segment.index.get_loc(segment.index[-1])
        buy_point_loc = segment.index.get_loc(buy_point.name)

        if (today_df_loc - buy_point_loc) < real_trade_point_and_buy_point_max_time_difference:

          print("We start at time " + str(segment.index[0]))
          print("We are now at time " + str(segment.index[-1]))
          print(signal)
          print("Below is buy point")
          print(buy_point)
          print()

          if buy_point.name not in buy_points.keys():
            buy_points[buy_point.name] = [segment.index[-1]]
          else:
            buy_points[buy_point.name].append(segment.index[-1])

          last_buy_point_dataset = segment
      except:
        pass
    
    sets_checked += 1
    if(sets_checked % 210 == 0):
      print("We have checked " + str(sets_checked) + " sets")
      print()

KeyboardInterrupt: 

In [20]:
real_trade_buy_points = []
first_real_trade_buy_points_time_index = None
trade_returns = []
avg_return = None

for key in buy_points.keys():
  buy_point_found_dates = buy_points[key]
  if len(buy_point_found_dates) > 1:
    real_trade_buy_point_time_index = buy_point_found_dates[1]
    real_trade_buy_points.append(Stock_Point(real_trade_buy_point_time_index, stock_price_hr.loc[real_trade_buy_point_time_index]["Close"]))

if len(real_trade_buy_points) > 0:
  first_real_trade_buy_points_time_index = real_trade_buy_points[0].time_index
  stock_price_after_trade = stock_price_hr[stock_price_hr.index > first_real_trade_buy_points_time_index]

  for i in range(len(stock_price_hr)):
    current_day_price = stock_price_hr.iloc[i, :]
    real_trade_buy_points_copy = real_trade_buy_points[:]
    for trade_point in real_trade_buy_points:
      if (current_day_price.name > trade_point.time_index):
        should_sell = trade_point.update_and_should_sell(current_day_price["Close"], current_day_price.name, stop_loss_schema = "half_sell_half_move")
        if should_sell or (i == (len(stock_price_hr) - 1)):
          trade_returns.append(trade_point.get_return(current_day_price["Close"]))
          print("We sell the trade we bought at " + str(trade_point.time_index) + " under price " + str(trade_point.original_close_price) + " at the date " + str(current_day_price.name) + " under price " + str(current_day_price["Close"]))
          real_trade_buy_points_copy.remove(trade_point)
    real_trade_buy_points = real_trade_buy_points_copy

if len(trade_returns) > 0:
  avg_return = sum(trade_returns)/len(trade_returns)

avg_return

We sell the trade we bought at 2020-01-19 08:00:00 under price 9077.03 at the date 2020-01-23 08:00:00 under price 8519.33
We sell half of the trade we bought at 2020-01-28 08:00:00 under price 9004.87 at the date 2020-01-30 17:00:00 under price 9472.92
We sell the trade we bought at 2020-01-28 08:00:00 under price 9004.87 at the date 2020-02-17 04:00:00 under price 9722.57
We sell the trade we bought at 2020-02-19 08:00:00 under price 10161.86 at the date 2020-02-19 23:00:00 under price 9601.99
We sell half of the trade we bought at 2020-04-24 01:00:00 under price 7480.6 at the date 2020-04-29 05:00:00 under price 7857.01
We sell half of the trade we bought at 2020-04-24 13:00:00 under price 7485.31 at the date 2020-04-29 06:00:00 under price 7939.74
We sell the trade we bought at 2020-04-24 01:00:00 under price 7480.6 at the date 2020-04-30 17:00:00 under price 8563.55
We sell the trade we bought at 2020-04-24 13:00:00 under price 7485.31 at the date 2020-04-30 17:00:00 under price 8

0.017447449140267156

# Test MACD and ZigZag vs Date
If our sampling size or number of date increases, the plot visualization will look differently, which is natural since there is a horizontal axis scale change. However, we wonder if the data values themselves will change or not.

In [ ]:
stock_price_hr_210 = stock.history(period="210d", interval='1h')
macd210 = macd_details_date_stripped_plot(stock_price_hr_210)

In [ ]:
zigzag210 = zigzag_date_stripped_plot(stock_price_hr_210)

In [ ]:
stock_price_hr_21_first = stock.history(start="2022-04-20", interval='1h')[:21*7]
macd21f = macd_details_date_stripped_plot(stock_price_hr_21_first)
zigzag21f = zigzag_date_stripped_plot(stock_price_hr_21_first)

In [ ]:
stock_price_hr_21_end = stock.history(period="21d", interval='1h')[:21*7]
macd21e = macd_details_date_stripped_plot(stock_price_hr_21_end)
zigzag21e = zigzag_date_stripped_plot(stock_price_hr_21_end)

In [ ]:
stock_price_hr_21_middle = stock.history(start="2022-05-17", interval='1h')[:21*7]
macd21m = macd_details_date_stripped_plot(stock_price_hr_21_middle)
zigzag21m = zigzag_date_stripped_plot(stock_price_hr_21_middle)

In [ ]:
np.all(np.equal(macd210[:147].to_numpy(), macd21f.to_numpy()))

True

In [ ]:
np.all(np.equal(zigzag210[:147], zigzag21f))

False

In [ ]:
zigzag210[:147]

array([445.80999756, 446.22168405, 446.63337054, 447.04505702,
       447.45674351, 447.86843   , 448.28011649, 448.69180298,
       446.8283488 , 444.96489461, 443.10144043, 441.23798625,
       439.37453206, 437.51107788, 435.6476237 , 433.78416951,
       431.92071533, 430.05726115, 428.19380697, 426.33035278,
       424.4668986 , 422.60344442, 420.73999023, 422.29199219,
       423.84399414, 425.39599609, 426.94799805, 428.5       ,
       426.93000031, 425.36000061, 423.79000092, 422.22000122,
       420.65000153, 419.08000183, 417.51000214, 415.94000244,
       419.0249939 , 422.10998535, 420.89749146, 419.68499756,
       418.47250366, 417.26000977, 419.17833964, 421.09666951,
       423.01499939, 424.93332926, 426.85165914, 428.76998901,
       427.04037476, 425.3107605 , 423.58114624, 421.85153198,
       420.12191772, 418.39230347, 416.66268921, 414.93307495,
       413.20346069, 411.47384644, 409.74423218, 408.01461792,
       406.28500366, 407.70469093, 409.1243782 , 410.54

In [ ]:
zigzag21f

array([445.80999756, 446.22168405, 446.63337054, 447.04505702,
       447.45674351, 447.86843   , 448.28011649, 448.69180298,
       446.8283488 , 444.96489461, 443.10144043, 441.23798625,
       439.37453206, 437.51107788, 435.6476237 , 433.78416951,
       431.92071533, 430.05726115, 428.19380697, 426.33035278,
       424.4668986 , 422.60344442, 420.73999023, 422.29199219,
       423.84399414, 425.39599609, 426.94799805, 428.5       ,
       426.93000031, 425.36000061, 423.79000092, 422.22000122,
       420.65000153, 419.08000183, 417.51000214, 415.94000244,
       419.0249939 , 422.10998535, 420.89749146, 419.68499756,
       418.47250366, 417.26000977, 419.17833964, 421.09666951,
       423.01499939, 424.93332926, 426.85165914, 428.76998901,
       427.04037476, 425.3107605 , 423.58114624, 421.85153198,
       420.12191772, 418.39230347, 416.66268921, 414.93307495,
       413.20346069, 411.47384644, 409.74423218, 408.01461792,
       406.28500366, 407.70469093, 409.1243782 , 410.54

In [ ]:
fig = px.line(x=range(147), y=[zigzag210[:147], zigzag21f])
fig.show()

In [ ]:
np.all(np.equal(macd210[-147:].to_numpy(), macd21e.to_numpy()))

False

In [ ]:
np.equal(macd210[-147:].to_numpy(), macd21e.to_numpy())

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [ ]:
np.all(np.equal(zigzag210[-147:], zigzag21e))

False

In [ ]:
macd210[-147:].to_numpy()

array([-1.58237291, -1.35721862, -1.1884834 , -1.07823863, -0.80128503,
       -0.46907135, -0.15079457,  0.37151062,  0.90572073,  1.3298952 ,
        1.71248382,  1.89459671,  1.94947715,  2.03168201,  1.97002779,
        2.00776404,  1.98652826,  1.96161043,  1.91335268,  1.86339192,
        1.77341801,  1.2591262 ,  0.94060036,  0.7275012 ,  0.71782686,
        0.7471383 ,  0.81383933,  0.88833253,  1.12164452,  1.1592419 ,
        1.2743971 ,  1.35169236,  1.45827105,  1.58897109,  1.74027416,
        1.91955242,  1.97990529,  2.08280236,  2.14766177,  2.25297636,
        2.35528351,  2.25383982,  2.00668215,  1.73911907,  1.4804891 ,
        1.2408055 ,  0.94544963,  0.63945442,  0.38923578,  0.35229159,
        0.35682497,  0.37944497,  0.41198755,  0.48464022,  0.48931689,
        0.70324551,  0.7918409 ,  0.82829876,  0.76855928,  0.7376135 ,
        0.71373857,  1.20940552,  1.54807019,  2.10528   ,  2.66622349,
        3.06901473,  3.3686913 ,  3.47734122,  3.4933227 ,  3.59

In [ ]:
macd21e.to_numpy()

array([ 0.00000000e+00,  3.14101195e-02,  3.10295791e-02,  1.40292257e-02,
        9.34325626e-02,  1.96279519e-01,  2.88754706e-01,  5.07822425e-01,
        7.32169821e-01,  8.90966102e-01,  1.03690611e+00,  1.06222942e+00,
        1.02003454e+00,  1.01982860e+00,  9.30110060e-01,  9.32826225e-01,
        9.01661411e-01,  8.78073724e-01,  8.44135122e-01,  8.15678095e-01,
        7.59369931e-01,  3.50552491e-01,  1.13447033e-01, -3.13896416e-02,
        6.14498633e-04,  6.51934041e-02,  1.61175361e-01,  2.62380374e-01,
        5.05533877e-01,  5.68927726e-01,  7.02531315e-01,  8.00268403e-01,
        9.24434935e-01,  1.07041675e+00,  1.23511588e+00,  1.42567655e+00,
        1.50322839e+00,  1.62095442e+00,  1.70208229e+00,  1.82167005e+00,
        1.93806835e+00,  1.85784927e+00,  1.63654990e+00,  1.39507142e+00,
        1.16167351e+00,  9.45966532e-01,  6.75112018e-01,  3.92921540e-01,
        1.64296309e-01,  1.43652942e-01,  1.62648545e-01,  1.98410584e-01,
        2.42999242e-01,  

As we can see, 21 days to now vs last 21 days from the data of 210 days to now have significantly different macd values. Therefore, we need to maintain a 21 day window sliding across the past data to examine buy points rather than operating on a 210 days dataset.

In [ ]:
zigzag210 = zigzag_date_stripped_plot(stock_price_hr_210)
zigzag_hr_21f = zigzag_date_stripped_plot(stock_price_hr_210[:147])

In [ ]:
np.all(np.equal(zigzag210[:147], zigzag_hr_21f))

False

In [ ]:
fig = px.line(x=range(147), y=[zigzag210[:147], zigzag_hr_21f])
fig.show()

# Reserve

In [ ]:
# def find_buy_points(signal):
#     state_up =True
#     up = signal[1] > signal[0]
#     prev_up = up
#     up_point = None
    
#     for i in range(1, len(signal) - 1):
#         up = signal[i] <= signal[i + 1]
#         if state_up:
#             if prev_up and not up:
#                 state_up = False
#                 up_point = i 
#         else:
#             if (not prev_up and up) and signal[i] > signal[up_point]: 
#                 return up_point, i
#         prev_up = up
#     return None